In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
from scipy.stats import ttest_rel

In [ ]:
# get raw data frame from csv
raw_data = pd.read_csv('/Users/erinburns/Library/CloudStorage/Box-Box/CoganLab/D_Data/GlobalLocal/combinedData.csv')

In [ ]:
# add columns for previous trial accuracy and previous trial type
raw_data['prev_acc'] = raw_data.groupby('subject_ID')['acc'].shift(1)
raw_data['prev_congruency'] = raw_data.groupby('subject_ID')['congruency'].shift(1)
raw_data['prev_switch'] = raw_data.groupby('subject_ID')['switchType'].shift(1)
raw_data['is_post_error'] = raw_data['prev_acc'] == 0

df = raw_data.copy()

In [ ]:
# set up dictionaries
post_iR_error_inc_correct = {
    'post_error': True,   
    'prev_cong': 'i',     
    'prev_sw': 'r',       
    'curr_cong': 'i',     
    'current_acc': 1      
}
post_iR_error_switch_correct = {
    'post_error': True,   
    'prev_cong': 'i',     
    'prev_sw': 'r',       
    'curr_sw': 's',     
    'current_acc': 1      
}
post_cS_error_switch_correct = {
    'post_error': True,   
    'prev_cong': 'c',     
    'prev_sw': 's',       
    'curr_sw': 's',     
    'current_acc': 1      
}
post_cS_error_inc_correct = {
    'post_error': True,   
    'prev_cong': 'c',     
    'prev_sw': 's',       
    'curr_cong': 'i',     
    'current_acc': 1      
}

In [ ]:
def get_subject_means(df, post_error=None, prev_cong=None, prev_sw=None, curr_cong=None, curr_sw=None, current_acc=1):
    """
    Filters data and returns mean RT per subject.
    post_error: True (post-error), False (post-correct), or None (both)
    prev_cong: 'i', 'c'
    prev_sw: 'r', 's', or None 
    curr_cong: 'i', 'c'
    curr_sw: 'r', 's', or None 
    current_acc: 1 (correct), 0 (incorrect)
    """
    mask = (df['RT'] > 0) # Base mask
    
    if post_error is not None:
        mask &= (df['is_post_error'] == post_error)
    if prev_cong is not None:
        mask &= (df['prev_cong'] == prev_cong)
    if prev_sw is not None:
        mask &= (df['prev_sw'] == prev_sw)
    if current_acc is not None:
        mask &= (df['acc'] == current_acc)

    if curr_cong is not None:
        mask &= (df['congruency'] == curr_cong)
    if curr_sw is not None:
        mask &= (df['switchType'] == curr_sw)
    if current_acc is not None:
        mask &= (df['acc'] == current_acc)
        
    # Group by subject and calculate mean + trial count
    stats = df[mask].groupby('subject_ID')['RT'].agg(['mean', 'count'])
    return stats

In [ ]:
def run_comparison(df, label1, label2, params1, params2):
    """
    params1 and params2 are dictionaries of the manual inputs we discussed.
    Example: params1 = {'post_error': True, 'curr_cong': 'i'}
    """
    # subject mean for both conditions
    res1 = get_subject_means(df, **params1)
    res2 = get_subject_means(df, **params2)
    
    # compare same subject
    comparison_df = pd.merge(res1, res2, on='subject_ID', suffixes=('_1', '_2'))
    
    # t-test
    t_stat, p_val = stats.ttest_rel(final['mean_1'], final['mean_2'])
    
    # print
    print(f"Comparison: {label1} vs {label2}")
    print(f"N Subjects: {len(final)}")
    print(f"Mean {label1}: {final['mean_1'].mean():.2f} ms")
    print(f"Mean {label2}: {final['mean_2'].mean():.2f} ms")
    print(f"t({len(final)-1}) = {t_stat:.3f}, p = {p_val:.4f}")
    
    return final